# UMAP

In [1]:
import sys
import seaborn as sns
import pandas as pd 
import numpy as np
from itertools import combinations
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import torch
import anndata as ad
import scanpy as sc
import os
import gc
from importlib import reload
print("C1")

from datasets import Dataset, load_from_disk
from datasets import load_dataset
from geneformer import EmbExtractor

from sklearn.cluster import KMeans
print("C2")

#classifer tools
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
print("C3")

sns.set_style('white')
torch.cuda.empty_cache()

import os

from datasets import Dataset, load_from_disk, load_dataset
import geneformer

from datetime import datetime
print("C4")
DEFAULT_NAME_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_name_id_dict.pkl"
DEFAULT_TOKEN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/token_dictionary.pkl"
DEFAULT_MEDIAN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_median_dictionary.pkl"

sns.set_style('white')
torch.cuda.empty_cache()
print("C5")

C1
C2
C3
C4
C5


# Importing the anndata files. 

In [2]:
## Iniitla cells
# Just pick from a random file where they were copied (the first half ish of them have them still in original). Then filter by type

harvest_file_path = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/2024-07-31_03-49-01_job_number_22.h5ad'
harvest_file_adata = ad.read_h5ad(harvest_file_path)
initial_adata = harvest_file_adata[harvest_file_adata.obs['type'] == 'initial']


# get adata up and working
# Ensure hsc_adata is already defined and populated with data

# 1. Check the contents of the original obs
print("Original obs contents:")
print(initial_adata.obs.head())

# 2. Create the backup
initial_adata.uns['obs_backup'] = initial_adata.obs.copy()

# 3. Verify the backup was created correctly
print("Contents of obs_backup after creation:")
print(initial_adata.uns['obs_backup'].head())

# 4. Clear the original obs
initial_adata.obs = pd.DataFrame(index=initial_adata.obs.index)

# 5. Check the contents of the cleared obs and the backup
print("Contents of cleared obs:")
print(initial_adata.obs.head())

print("Contents of obs_backup after clearing original obs:")
print(initial_adata.uns['obs_backup'].head())



#initial_adata.obs['kmeans_cluster'] = initial_kmeans.labels_
initial_adata.obs['recipe_name'] = np.nan
initial_adata.obs['recipe_number'] = np.nan
initial_adata.obs['cell_id'] = initial_adata.uns['obs_backup']['cell_id']
initial_adata.obs['type'] = 'initial'



initial_adata.obs.sample(10)



Original obs contents:
                                   input_ids   cell_type         dataset  \
0  [16345  9009 13048 ...  9332 13451  5456]  fibroblast  TS_Vasculature   
1  [12119  9190 16876 ... 11232  1132  1022]  fibroblast  TS_Vasculature   
2  [ 3878  9009  4115 ...  4697 10362 12098]  fibroblast  TS_Vasculature   
3  [ 3878 16916 18367 ... 15470  3946  2153]  fibroblast  TS_Vasculature   
4  [ 6196 16916 10920 ...  7629 10148 17125]  fibroblast  TS_Vasculature   

  length      ignore standardized_cell_type  broad_type __index_level_0__  \
0   2048  fibroblast             Fibroblast  fibroblast            109770   
1   2048  fibroblast             Fibroblast  fibroblast            109771   
2   2048  fibroblast             Fibroblast  fibroblast            109774   
3   2048  fibroblast             Fibroblast  fibroblast            109776   
4   2048  fibroblast             Fibroblast  fibroblast            109777   

  cell_id recipe     type  
0  cell_1    raw  initial  
1

/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_3489677/1375448826.py:17: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  initial_adata.uns['obs_backup'] = initial_adata.obs.copy()


,recipe_name,recipe_number,cell_id,type
7100,NaN,NaN,cell_7101,initial
4782,NaN,NaN,cell_4783,initial
11108,NaN,NaN,cell_11109,initial
5812,NaN,NaN,cell_5813,initial
5507,NaN,NaN,cell_5508,initial
13029,NaN,NaN,cell_13030,initial
9962,NaN,NaN,cell_9963,initial
6722,NaN,NaN,cell_6723,initial
15070,NaN,NaN,cell_15071,initial
11514,NaN,NaN,cell_11515,initial


# ISP Cells

In [3]:
file = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/' + '2024-07-31_03-17-04_job_number_12.h5ad'
reprog_centroid_adata_wid = ad.read_h5ad(reprog_wid_path)
# this is not actually written into the file idk if we care? 
reprog_centroid_adata_wid.obs['type'] = 'reprogrammed'
reprog_centroid_adata_wid.obs['cluster_number'] = range(1, len(reprog_centroid_adata_wid.obs) + 1)
kmeans = KMeans(n_clusters=10, random_state=0)

# Fit KMeans to the DataFrame (make sure to use the feature columns only)
reprog_centroid_adata['kmeans'] = kmeans.fit_predict(reprog_centroid_adata)

reprog_centroid_adata_wid.obs.sample(10)



NameError: name 'reprog_wid_path' is not defined

In [4]:
from itertools import combinations
import pandas as pd
import anndata as ad
import numpy as np
from sklearn.cluster import KMeans

# Initialize an empty AnnData object (This will be replaced by actual data)
reprog_centroid_adata = ad.AnnData()

all_cluster_labels = []
all_cell_id_labels = []
all_recipe_name_labels = []
all_recipe_num_labels = []

def ten_choose_five():
    gene_list = [
        'GATA2', 
        'GFI1B', 
        'FOS', 
        'STAT5A',
        'REL',
        'FOSB',
        'IKZF1',
        'RUNX3',
        'MEF2C',
        'ETV6',
    ]
    len_sublist = 5
    sublists = list(combinations(gene_list, len_sublist))
    
    df = pd.DataFrame({
        'recipe_iteration': range(1, len(sublists) + 1),
        'recipe_list': [list(sublist) for sublist in sublists]
    })
    return df

pert_df = ten_choose_five()
pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

print("Going to read in ")
print(file)
one_recipe_adata = ad.read_h5ad(file)
print(one_recipe_adata.obs.columns)

one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

# Perform k-means clustering
kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

# Collect data for the AnnData object
all_cell_id_labels = one_recipe_adata.obs['cell_id'].tolist()
all_cluster_labels = one_recipe_adata.obs['kmeans_clusters'].tolist()

recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
if len(recipe_names_this_file) == 1:
    recipe_as_string = recipe_names_this_file[0]
    recipe_as_list = recipe_as_string.split(';')
    print(f'The unique value is: {recipe_as_list}')
    recipe_num = pert_dict[recipe_as_string]
    print(recipe_num)
else:
    print('Error: There are multiple unique values or no values in the column.')

# Prepare the AnnData object with all cells
reprog_centroid_adata = ad.AnnData(X=one_recipe_adata.X)
reprog_centroid_adata.obs['kmeans_cluster'] = all_cluster_labels
reprog_centroid_adata.obs['cell_id'] = all_cell_id_labels
reprog_centroid_adata.obs['recipe_name'] = [recipe_as_string] * len(all_cluster_labels)
reprog_centroid_adata.obs['recipe_number'] = [recipe_num] * len(all_cluster_labels)

print("All cell data and kmeans_cluster values have been added to reprog_centroid_adata.")
print(reprog_centroid_adata.obs.head(12))


Going to read in 
/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/2024-07-31_03-17-04_job_number_12.h5ad


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['input_ids', 'cell_type', 'dataset', 'length', 'ignore',
       'standardized_cell_type', 'broad_type', '__index_level_0__', 'cell_id',
       'recipe', 'type'],
      dtype='object')
The unique value is: ['GATA2', 'GFI1B', 'FOS', 'FOSB', 'IKZF1']
12
All cell data and kmeans_cluster values have been added to reprog_centroid_adata.
    kmeans_cluster  cell_id                 recipe_name  recipe_number
0                8   cell_1  GATA2;GFI1B;FOS;FOSB;IKZF1             12
1                5   cell_2  GATA2;GFI1B;FOS;FOSB;IKZF1             12
2                6   cell_3  GATA2;GFI1B;FOS;FOSB;IKZF1             12
3                0   cell_4  GATA2;GFI1B;FOS;FOSB;IKZF1             12
4                6   cell_5  GATA2;GFI1B;FOS;FOSB;IKZF1             12
5                4   cell_6  GATA2;GFI1B;FOS;FOSB;IKZF1             12
6                6   cell_7  GATA2;GFI1B;FOS;FOSB;IKZF1             12
7                4   cell_8  GATA2;GFI1B;FOS;FOSB;IKZF1             12
8                0   

/tmp/ipykernel_3489677/3842848350.py:50: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_


In [5]:
from itertools import combinations
import pandas as pd
import anndata as ad
import numpy as np
from sklearn.cluster import KMeans
import os

def ten_choose_five():
    gene_list = [
        'GATA2', 
        'GFI1B', 
        'FOS', 
        'STAT5A',
        'REL',
        'FOSB',
        'IKZF1',
        'RUNX3',
        'MEF2C',
        'ETV6',
    ]
    len_sublist = 5
    sublists = list(combinations(gene_list, len_sublist))
    
    df = pd.DataFrame({
        'recipe_iteration': range(1, len(sublists) + 1),
        'recipe_list': [list(sublist) for sublist in sublists]
    })
    return df

pert_df = ten_choose_five()
pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# List of files to process
file_list = ['/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-03-23_job_number_89.h5ad','/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-01-47_job_number_88.h5ad']  # Replace with actual file names

# Initialize a list to collect data
all_data = []

for file in file_list:
    print("Going to read in ", file)
    one_recipe_adata = ad.read_h5ad(file)
    print(one_recipe_adata.obs.columns)

    one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
    one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

    # Extract recipe number
    recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
    if len(recipe_names_this_file) == 1:
        recipe_as_string = recipe_names_this_file[0]
        recipe_num = pert_dict.get(recipe_as_string, None)
        if recipe_num is None:
            print(f"Recipe number not found for {recipe_as_string}")
            continue
    else:
        print('Error: There are multiple unique values or no values in the column.')
        continue

    # Combine recipe number with kmeans cluster number
    one_recipe_adata.obs['kmeans'] = one_recipe_adata.obs['kmeans_clusters'].apply(
        lambda x: int(f"{recipe_num}{x}")
    )

    # Collect data
    all_data.append(one_recipe_adata)

# Concatenate all AnnData objects
combined_adata = ad.concat(all_data, join='outer')

print("Cluster counts:")
combined_adata.obs.sample(20)

# she works b"h


Going to read in  /nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-03-23_job_number_89.h5ad


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Index(['input_ids', 'cell_type', 'dataset', 'length', 'ignore',
       'standardized_cell_type', 'broad_type', '__index_level_0__', 'cell_id',
       'recipe', 'type'],
      dtype='object')


/tmp/ipykernel_3489677/235025320.py:49: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_


Going to read in  /nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-01-47_job_number_88.h5ad


...

Index(['input_ids', 'cell_type', 'dataset', 'length', 'ignore',
       'standardized_cell_type', 'broad_type', '__index_level_0__', 'cell_id',
       'recipe', 'type'],
      dtype='object')


/tmp/ipykernel_3489677/235025320.py:49: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_


Cluster counts:


/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


,input_ids,cell_type,dataset,length,ignore,standardized_cell_type,broad_type,__index_level_0__,cell_id,recipe,type,kmeans_clusters,kmeans
26370,"[14409, 12698, 15641, 404, 1532, 454, 16916, 9...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,162764,cell_11063,GATA2;FOS;IKZF1;RUNX3;MEF2C,reprogrammed,9,889
21338,"[14409, 12698, 15641, 404, 7725, 16916, 16876,...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,152056,cell_6031,GATA2;FOS;IKZF1;RUNX3;ETV6,reprogrammed,2,892
29437,"[14409, 12698, 15641, 404, 1532, 16916, 454, 6...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,169456,cell_14130,GATA2;FOS;IKZF1;RUNX3;MEF2C,reprogrammed,9,889
28572,"[14409, 12698, 15641, 404, 7725, 9009, 16166, ...",Fibroblasts,TS_Fat,1657,Fibroblasts,Fibroblast,fibroblast,167541,cell_13265,GATA2;FOS;IKZF1;RUNX3;ETV6,reprogrammed,0,890
29277,"[14409, 12698, 15641, 404, 1532, 9009, 20760, ...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,169107,cell_13970,GATA2;FOS;IKZF1;RUNX3;MEF2C,reprogrammed,9,889
15792,"[14409, 12698, 15641, 404, 7725, 9009, 12119, ...",fibroblast,TS_Vasculature,2048,fibroblast,Fibroblast,fibroblast,111136,cell_485,GATA2;FOS;IKZF1;RUNX3;ETV6,reprogrammed,0,890
22150,"[14409, 12698, 15641, 404, 1532, 16916, 3878, ...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,153805,cell_6843,GATA2;FOS;IKZF1;RUNX3;MEF2C,reprogrammed,9,889
23090,"[14409, 12698, 15641, 404, 7725, 454, 15960, 1...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,155805,cell_7783,GATA2;FOS;IKZF1;RUNX3;ETV6,reprogrammed,5,895
22694,"[14409, 12698, 15641, 404, 1532, 9190, 5716, 3...",Myofibroblasts,TS_Fat,2048,Myofibroblasts,Fibroblast,fibroblast,154992,cell_7387,GATA2;FOS;IKZF1;RUNX3;MEF2C,reprogrammed,5,885
27355,"[14409, 12698, 15641, 404, 1532, 16916, 3878, ...",Fibroblasts,TS_Fat,2048,Fibroblasts,Fibroblast,fibroblast,164960,cell_12048,GATA2;FOS;IKZF1;RUNX3;MEF2C,reprogrammed,0,880


In [ ]:
# ok here's where we have to be careful.  going back to an old version of the file to find where reprog_centroid_adata_wid was made, and if there's composite of the original cells. checking in turbo first


In [7]:
import os
import re

def list_reprogramming_jobs(directory, start_job_number, end_job_number):
    """
    List and sort files in the given directory by job number within the specified range.

    Parameters:
    directory (str): The directory containing the files.
    start_job_number (int): The starting job number.
    end_job_number (int): The ending job number.

    Returns:
    list: A sorted list of file names within the specified job number range.
    """
    # List all files in the directory
    files = os.listdir(directory)

    # Define a regex pattern to extract job numbers
    pattern = re.compile(r'_job_number_(\d+)\.h5ad$')

    # Create a list of tuples (job_number, file_name)
    files_with_job_numbers = []
    for file in files:
        match = pattern.search(file)
        if match:
            job_number = int(match.group(1))
            if start_job_number <= job_number <= end_job_number:
                files_with_job_numbers.append((job_number, file))

    # Sort files by job number
    files_with_job_numbers.sort(key=lambda x: x[0])

    # Extract sorted file names
    sorted_files = [file for _, file in files_with_job_numbers]

    return sorted_files



directory = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/'
files_in_directory = list_reprogramming_jobs(directory, 1, 252)
files_in_directory = directory + files_in_directory[file] for file in files_in_directory
files_in_directory

SyntaxError: invalid syntax (3849309790.py, line 43)

In [ ]:
from itertools import combinations
import pandas as pd
import anndata as ad
import numpy as np
from sklearn.cluster import KMeans
import os

def ten_choose_five():
    gene_list = [
        'GATA2', 
        'GFI1B', 
        'FOS', 
        'STAT5A',
        'REL',
        'FOSB',
        'IKZF1',
        'RUNX3',
        'MEF2C',
        'ETV6',
    ]
    len_sublist = 5
    sublists = list(combinations(gene_list, len_sublist))
    
    df = pd.DataFrame({
        'recipe_iteration': range(1, len(sublists) + 1),
        'recipe_list': [list(sublist) for sublist in sublists]
    })
    return df

pert_df = ten_choose_five()
pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# List of files to process
  # Replace with actual file names

# Initialize a list to collect data
all_data = []

for file in files_in_directory:
    print("Going to read in ", file)
    one_recipe_adata = ad.read_h5ad(file)
    print(one_recipe_adata.obs.columns)

    one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
    one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

    # Extract recipe number
    recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
    if len(recipe_names_this_file) == 1:
        recipe_as_string = recipe_names_this_file[0]
        recipe_num = pert_dict.get(recipe_as_string, None)
        if recipe_num is None:
            print(f"Recipe number not found for {recipe_as_string}")
            continue
    else:
        print('Error: There are multiple unique values or no values in the column.')
        continue

    # Combine recipe number with kmeans cluster number
    one_recipe_adata.obs['kmeans'] = one_recipe_adata.obs['kmeans_clusters'].apply(
        lambda x: int(f"{recipe_num}{x}")
    )

    # Collect data
    all_data.append(one_recipe_adata)

# Concatenate all AnnData objects
combined_adata = ad.concat(all_data, join='outer')

print("Cluster counts:")
combined_adata.obs.sample(20)

# she works b"h


In [ ]:
# forgot to write in the kmeans column so we r doing that now


In [ ]:
adata = ad.AnnData()

for file in files_in_directory:
    this_recipe_adata = pd.read_h5ad(file)
    this_recipe_adata = this_recipe_adata[this_recipe_adata.obs['type'] == 'reprogrammed']
    
    adata = ad.concat([adata, this_recipe_adata], join='outer')
    adata.write('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/perturbed_combined.h5ad', compression='gzip')